<a href="https://colab.research.google.com/github/y-arjun-y/liars-miller-rabin/blob/main/liars_miller_rabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Implementation of the Miller-Rabin primality test in Python3. 
Function by https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python:_Proved_correct_up_to_large_N.
Variables and formula is based on https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test.
A return value of False means n in not prime.
A return value of True means n is very likely to be a prime.
'''

def miller_rabin(n, a):
    # checking if it is an integer
    if n != int(n):
        return False

    n = int(n)

    # base cases
    if n == 0 or n == 1 or n == 4 or n == 6 or n == 8 or n == 9:
        return False

    if n == 2 or n == 3 or n == 5 or n == 7:
        return True

    if n % 2 == 0:
        return False

    # assigning variables
    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1

    # trial run
    def trial_composite(a):
        if pow(a, d, n) == 1:
            return False

        for i in range(s):
            if pow(a, 2**i * d, n) == n - 1:
                return False
        return True

    # number of trials, directly proportional to time and accuracy.
    num_trials = 10

    # run
    for _ in range(num_trials):
        if trial_composite(a):
            return False

    return True

In [ ]:
'''
Returns the length of prime numbers of a given range.
'''

def len_probable_prime(int, a):
    results = []

    # main loop
    for i in range(int + 1):
        if miller_rabin(i, a) == True:
            results.append(miller_rabin(i, a))

    return len(results)

In [ ]:
# imports
from tqdm import tqdm

# constants
DEFENDANT = 1000
WITNESS_RANGE = 100
ACTUAL_LEN_PRIMES = 168

# witness variables
excellent_witnesses = []  # = ACTUAL_LEN_PRIMES
okay_witnesses = []  # ±5 ACTUAL_LEN_PRIMES
false_witnessses = []  # != ACTUAL_LEN_PRIMES
worst_witness = []  # biggest difference
witnesses = [i for i in range(WITNESS_RANGE + 1)]

# main loop
for i in tqdm(range(WITNESS_RANGE + 1)):
    if len_probable_prime(DEFENDANT, i) == ACTUAL_LEN_PRIMES:
        excellent_witnesses.append(i)
    elif len_probable_prime(DEFENDANT, i) - ACTUAL_LEN_PRIMES <= 5:
        okay_witnesses.append(i)

    if len_probable_prime(DEFENDANT, i) != ACTUAL_LEN_PRIMES:
        false_witnessses.append(i)

100%|██████████| 101/101 [00:03<00:00, 25.50it/s]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
